# Literature Metadata Retrieval

Chemical and biological literature are stored in the repositories of various publishers. There are many services, such as CrossRef and PubMed, that assign accession numbers to literature across many repositories and aggregate their metadata.

This notebook explores some of the ways to retrieve metadata about a document given one of these services' identifiers.

In [1]:
import requests

## Digital Object Identifier System

The goal of the [Digital Object Identifier (DOI)](http://www.doi.org/) system is to assign unique identifiers not just to literature, but to all digital documents. Taking inspiration from other cataloging systems such as the ISBN, there is some organization and modularity to the identifiers.

In [2]:
doi_ex = "10.1186/2041-1480-5-5"

### DOI Service

This website provides a URL resolution service at http://dx.doi.org/. For example, http://dx.doi.org/10.1186/2041-1480-5-5 will resolve to the Journal of Biomedical Semantics' splash page for the corresponding article about the results from a BioHackathon in 2011 and 2012.

The website also provides a metadata service at http://doi.org/api/handles/. There is some, albeit sparse, [API documentation](https://www.doi.org/factsheets/DOIProxy.html).

In [3]:
doi_endpoint = "http://doi.org/api/handles/"

In [4]:
res = requests.get(doi_endpoint + doi_ex).json()
res

{'handle': '10.1186/2041-1480-5-5',
 'responseCode': 1,
 'values': [{'data': {'format': 'string',
    'value': 'http://www.jbiomedsem.com/content/5/1/5'},
   'index': 1,
   'timestamp': '2014-04-07T20:05:11Z',
   'ttl': 86400,
   'type': 'URL'},
  {'data': {'format': 'string', 'value': '20140407165738'},
   'index': 700050,
   'timestamp': '2014-04-07T20:05:11Z',
   'ttl': 86400,
   'type': '700050'},
  {'data': {'format': 'admin',
    'value': {'handle': '0.na/10.1186',
     'index': 200,
     'permissions': '111111110010'}},
   'index': 100,
   'timestamp': '2014-04-07T20:05:11Z',
   'ttl': 86400,
   'type': 'HS_ADMIN'}]}

### CrossRef 

CrossRef is a much more powerful metadata and citation indexing system built on top of the DOI system. Because citation information is generally inconsistient across the centuries of publications that various repositories serve, the data from this service must be handled carefully and checked for thoroughness. 

CrossRef has thorouch [API Documentation](https://github.com/CrossRef/rest-api-doc/blob/master/rest_api.md) on GitHub.

CrossRef also has a blog where they're developing new and interesting things! http://labs.crossref.org/

In [5]:
crossref_endpoint = "http://api.crossref.org/works/"

In [6]:
res = requests.get(crossref_endpoint + doi_ex).json()
res = res['message']
d = {}
d['publisher'] = res['publisher']
d['publication'] = max(res['container-title'], key=len)
d['author'] = "{}, {}".format(res['author'][0]['family'], res['author'][0]['given'])
d['publication_date'] = res['published-print']['date-parts'][0]
if 'subject' in res:
    d['subject'] = res['subject'][0]
if 'title' in res:
    d['title'] = res['title'][0]
d

{'author': 'Katayama, Toshiaki',
 'publication': 'Journal of Biomedical Semantics',
 'publication_date': [2014],
 'publisher': 'Springer Science + Business Media',
 'title': 'BioHackathon series in 2011 and 2012: penetration of ontology and linked data in life science domains'}

## PubMed

PubMed Identifiers (PMID) are an alternate accession given to articles in MEDLINE by the [PubMed](http://www.ncbi.nlm.nih.gov/pubmed) service. Unfortunately, the PubMed API and its [documentation](http://www.ncbi.nlm.nih.gov/home/api.shtml) are not so straightforwards. 

In [7]:
pmid_ex = "25081398"

### Entrez Database Search

Fred Trotter made a nice write-up on how to use it on his [blog](http://www.fredtrotter.com/2014/11/14/hacking-on-the-pubmed-api/). The Entrez Database doesn't always have an option to return json, so this solution uses the xml library in python and an XPath query to traverse the document returned by the efetch service to get an abstract. 

The XML results from this example can be viewed [here](http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&rettype=abstract&id=25081398).

In [8]:
entrez_endpoint = "http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&rettype=abstract&id="

In [9]:
import xml.etree.ElementTree as ET
res = requests.get(entrez_endpoint + pmid_ex)
root = ET.fromstring(res.content)

d = {}
d['title'] = root.find("./PubmedArticle/MedlineCitation/Article/ArticleTitle").text
d['publication'] = root.find("./PubmedArticle/MedlineCitation/Article/Journal/Title").text
d['abstract'] = root.find("./PubmedArticle/MedlineCitation/Article/Abstract/AbstractText").text
d

{'abstract': 'Current therapies for cancer have improved life expectancy of patients. Breast cancer and lymphoma survivors in up to 26% of cases can develop complications as a consequence of the chemotherapeutic and radiotherapeutic treatments. Echocardiography is a noninvasive method that can in all stages of cancer treatment perform a comprehensive evaluation and detect coronary, myocardial, valve and pericardial disease complications secondary to the therapeutic regimen used (radiotherapy and/or chemotherapy). Three-dimensional echocardiography derived left ventricular ejection fraction (LVEF) has an excellent correlation with cardiac magnetic resonance imaging and can be used to monitor LVEF; 2-dimensional speckle tracking echocardiography (2D-STE) derived strain and strain rate can detect changes in myocardial mechanics before changes in LVEF occur and can predict a future decrease in ejection fraction to less than 50% or of greater than 10% indicative of cardiotoxicity. Echocardi

### AltMetric

AltMetric already solved all of the problems with retreiving, curating, and publishing literature metadata, but it's limited for free users. They have a DOI endpoint at http://api.altmetric.com/v1/doi and a PMID endpoing at http://api.altmetric.com/v1/pmid.

In [10]:
altmetric_endpoint = "http://api.altmetric.com/v1/pmid/"

In [11]:
res = requests.get(altmetric_endpoint + pmid_ex)
res.content

b'Not Found'